# Dummy EAS reconstruction

This is a simple example on how GRANDlib classes can be used for data I/O in the case of shower parameters reconstruction. The traces used do not represent any real EAS traces, and the reconstruction used is completely meaningless, serving only ilustrational purpose.

First, let's generate a ROOT file with random, dummy events. The old file will be cleared if exists. This uses an example script examples/io/DataStoringExample.py. Please be sure to restart kernel each time you try this script.


In [1]:
import os.path

if os.path.isfile("dummy_data.root"):
    os.remove("dummy_data.root")
# Can't use %run - leaves stuff in memory that is needed empty later
#%run ../io/DataStoringExample.py dummy_data.root
os.system("../dataio/data_storing.py dummy_data.root")

4 traces for event 0
3 traces for event 1
5 traces for event 2
6 traces for event 3
4 traces for event 4
6 traces for event 5
3 traces for event 6
5 traces for event 7
3 traces for event 8
5 traces for event 9
Wrote trun
Wrote tadccounts
Wrote trawvoltage
Wrote tvoltage
Wrote tefield
Wrote tshower
Finished writing file dummy_data.root


0

Now we import the necessary python modules

In [2]:
import sys
import os
from grand.grandlib_classes.grandlib_classes import *
import grand.grandlib_classes.grandlib_classes as g

Let's create an Event, and provide the file name, run and event numbers (which we know to exist) for it

In [3]:
e = Event()
e.file = "dummy_data.root"
e.run_number = 0
e.event_number = 0

The Event has to be told to initialise its contents from the provided file, with provided run and event numbers.
We tell the Event that the tshower it reads from the file contains simulated (not reconstructed) values.

In [4]:
e.fill_event_from_trees(simshower=True)

Run information loaded.
Voltage information loaded.
Efield information loaded.
Shower information loaded.


Just for checking - what are the contents of the beginning of the first trace X of the Efield?

In [5]:
print(e.efields[0].trace.x[:10])

[-0.00231372 -0.00025708 -0.00012854 -0.00179956 -0.00089978 -0.00051416
  0.00231372  0.00154248  0.0006427   0.00167102]


In [6]:
print(e.simshower.energy_em, e.simshower.Xmax, e.simshower.azimuth, e.simshower.zenith)

52009656.0 135.80357 30.963005 77.99881


Initialise the reconstruction shower in the Event:

In [7]:
e.shower = Shower()

Storing the totally meaningless energy "reconstruction" in the Event:

In [8]:
e.shower.energy_em = 10**21*(np.max([np.max(np.sqrt(np.array(ef.trace.x)**2+np.array(ef.trace.y)**2+np.array(ef.trace.z)**2)) for ef in e.efields]))
e.shower.energy_primary = 1.2*e.shower.energy_em

Storing the totally meaningless Xmax "reconstruction" in the Event:

In [9]:
e.shower.Xmax = 1e6*(np.max([np.max(np.sqrt(np.array(ef.trace.x)**2+np.array(ef.trace.y)**2+np.array(ef.trace.z)**2)) for ef in e.efields]))

e.shower.Xmaxpos.x = 1e5*np.sum([np.sum(ef.trace.x) for ef in e.efields])
e.shower.Xmaxpos.y = 1e5*np.sum([np.sum(ef.trace.y) for ef in e.efields])
e.shower.Xmaxpos.z = 1e5*np.sum([np.sum(ef.trace.z) for ef in e.efields])

Storing the totally meaningless angle reconstruction in the Event:

In [10]:
e.shower.zenith = np.arccos(e.shower.Xmaxpos.z/np.sqrt(e.shower.Xmaxpos.x**2+e.shower.Xmaxpos.y**2+e.shower.Xmaxpos.z**2))
e.shower.azimuth = np.sign(e.shower.Xmaxpos.y)*np.arccos(e.shower.Xmaxpos.x/np.sqrt(e.shower.Xmaxpos.x**2+e.shower.Xmaxpos.y**2))

Write the reconstructed shower to another file:

In [11]:
if os.path.isfile("shower_reco.root"):
    os.remove("shower_reco.root")

#e.write_shower("shower_reco.root")
e.write_shower("dummy_data.root", tree_name="tshower_reco")

No valid tshower_reco TTree in the file dummy_data.root. Creating a new one.
